In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torchvision
from ast import literal_eval
from torch.utils.data.dataset import Dataset
from Source.DataLoading.DataProcessor import DataProcessor
from scipy.interpolate import interp1d
import torch
import pathlib
from torch.utils.data.dataset import Dataset
import requests

wqi_df = pd.read_csv('Data/DSVersion1Scalled.csv')

C:\Users\rmsry\AppData\Local\Temp\ipykernel_29544\1566710361.py:11: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [20]:
wqi_df['scaled_features_array'] = wqi_df['scaled_features'].apply(literal_eval)
wqi_df['scaled_features_numpy_array'] = wqi_df['scaled_features_array'].apply(np.array)
X = np.vstack(wqi_df.scaled_features_numpy_array.to_numpy())
Y = wqi_df.ClassificationStatus_index.to_numpy()

In [29]:
split_idx = int(0.8 * X.shape[0])

In [30]:
X_train = X[:split_idx]
X_test = X[split_idx:]

Y_train = Y[:split_idx]
Y_test = Y[split_idx:]

In [34]:
wqi_df.ClassificationStatus_index.nunique()

5

In [33]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((8396, 29), (8396,), (2099, 29), (2099,))

In [42]:
class ObjectView(object):
    def __init__(self, d):
        self.__dict__ = d

In [43]:
def get_dataset_args(as_dict=False):
    arg_dict = {
        "num_samples": 1000,  # modified for meta-learning
        "train_split": 0.8,
        "template_len": 12,
        "padding": [36, 60],
        "scale_coeff": 0.4,
        "max_translation": 48,
        "corr_noise_scale": 0.25,
        "iid_noise_scale": 2e-2,
        "shear_scale": 0.75,
        "final_seq_length": 40,
        "url": "https://github.com/greydanus/mnist1d/raw/master/mnist1d_data.pkl",
    }
    return arg_dict if as_dict else ObjectView(arg_dict)

In [44]:
class WQIDataset(Dataset):
    """Pytorch custom dataset for loading WQI dataset"""

    def __init__(
        self,
        root_path,
        train,
        **kwargs
    ):
        """
        Args:
            root_path (string): path to the dataset pickle file
            train (bool): selects between training and testing dataset
            regenerate (bool): Whether to regenerate the dataset
            download (bool): Downloads original 1D mnist dataset
            transform (Function): Applies transformation to PIL image
            target_transform (Function): Applies transformation to target values
        """
        self.args = get_dataset_args()
        self.path = root_path + "/DSVersion1Scalled.csv"
        self.train = train


        wqi_df = pd.read_csv(self.path)
        wqi_df['scaled_features_array'] = wqi_df['scaled_features'].apply(literal_eval)
        wqi_df['scaled_features_numpy_array'] = wqi_df['scaled_features_array'].apply(np.array)
        X = np.vstack(wqi_df.scaled_features_numpy_array.to_numpy())
        Y = wqi_df.ClassificationStatus_index.to_numpy()
        split_idx = int(0.8 * X.shape[0])
        X_train = X[:split_idx]
        X_test = X[split_idx:]

        Y_train = Y[:split_idx]
        Y_test = Y[split_idx:]

        self.dataset = {
            "x": X_train,
            "y": Y_train,
            "x_test": X_test,
            "y_test": Y_test,
        }        
        
        if train:
            self.imgs = self.dataset["x"]
            self.targets = self.dataset["y"]
        else:
            self.imgs = self.dataset["x_test"]
            self.targets = self.dataset["y_test"]

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        img, target = self.imgs[idx].astype("float32"), int(self.targets[idx])
        return img, target

In [45]:
class WQI(DataProcessor):
    """WQI dataset and manipulation tools"""

    def __init__(self, options):
        super().__init__(options)
        self.input_size = 8396
        self.output_size = 5

    def process(self):
        """Prepare WQi for use by the network"""
        train_loader = torch.utils.data.DataLoader(
            WQIDataset(
                root_path=self.options.input_data,
                train=True
            ),
            **self.options.kwargs
        )

        test_loader = torch.utils.data.DataLoader(
            WQIDataset(
                root_path=self.options.input_data,
                train=False
            ),
            **self.options.kwargs
        )
        return train_loader, test_loader

    def fc_reshape(self, data: torch.Tensor) -> torch.Tensor:
        """Reshapes MNIST data for FC use

        Args:
            data (torch.Tensor): input data

        Returns:
            torch.Tensor: reshaped data
        """
        return data.reshape(-1, 28 * 28)